In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scikit-learn nos ofrece una variedad ampliada de modelos Naive Bayes, para este problema usamos MultinomialNB que es pensado para este tipo de problemas
from sklearn.naive_bayes import MultinomialNB   

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

1) ¿Cuáles son las 10 palabras más encontradas en correos con SPAM y en correos No SPAM? ¿Hay palabras en común? ¿Algunas llaman la atención?

In [46]:
# Lee el archivo CSV
df = pd.read_csv('spambase.csv')
resultado = []
spam = ['NO SPAM', 'SPAM']
for choise in [0,1]:

    spam_df = df[df['spam'] == choise]

    word_freq_columns = [col for col in spam_df.columns if col.startswith('word_freq_')]

    suma_columnas = spam_df[word_freq_columns].sum()

    top_10 = suma_columnas.sort_values(ascending=False).head(10)

    resultado.append(top_10.to_dict())

    print('\n',spam[choise])
    for columna, valor in resultado[choise].items():
        print(f"{columna[10:]} : {valor}")

print("\n")
claves_iguales = set(resultado[0].keys()).intersection(set(resultado[1].keys()))
for key in claves_iguales:
    print(f"La palabra '{key[10:]}' se repite en ambos grupos de correo")



 NO SPAM
you : 3541702
george : 3527559
hp : 2496576
will : 1495268
your : 1223098
hpl : 1204398
re : 1159138
edu : 800669
address : 681569
meeting : 604460

 SPAM
you : 4105599
your : 2502597
will : 997100
free : 939790
our : 931799
all : 732080
mail : 635470
email : 578759
business : 521250
remove : 499309


La palabra 'will' se repite en ambos grupos de correo
La palabra 'your' se repite en ambos grupos de correo
La palabra 'you' se repite en ambos grupos de correo


2) Separe el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba (70% y 30% respectivamente).

In [47]:
# Obtenemos los atributos y target
X = (df.drop(columns="spam") * 100).astype(int)
#X = dataset2.drop(columns="spam")
y = df["spam"]

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3)


3) Utilizando un clasificador de Bayes ingenuo, entrene con el conjunto de entrenamiento.

In [57]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report

naives_bayes_student = CategoricalNB()
naives_bayes_student.fit(X_train, y_train)

y_pred = naives_bayes_student.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")

# Generar un reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

IndexError: index 510000 is out of bounds for axis 1 with size 434001

4) Utilizando un clasificador de Regresión Logística, entrene con el conjunto de entrenamiento (en este caso, normalice los datos).

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lo transformamos en DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

classifier_balance = LogisticRegression(random_state = 0, class_weight="balanced")
classifier_balance.fit(X_train_scaled, y_train)

y_pred = classifier_balance.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")

# Generar un reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))


Precisión del modelo: 0.83
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.74      0.85       845
           1       0.71      0.98      0.82       536

    accuracy                           0.83      1381
   macro avg       0.84      0.86      0.83      1381
weighted avg       0.87      0.83      0.84      1381



5) Calcule la matriz de confusión del conjunto de evaluación para ambos modelos. ¿Qué tipo de error comete más cada modelo? ¿Cuál de los dos tipos de error crees que es más importante para este problema?